***GENERATED CODE FOR smartfarmingsensordataforyieldprediction PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'farm_id', 'transformation_label': 'String Indexer'}], 'feature': 'farm_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'FARM0001', 'max': 'FARM0500', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'farm_id'}, {'feature_label': 'farm_id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('farm_id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'region', 'transformation_label': 'String Indexer'}], 'feature': 'region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Central USA', 'max': 'South USA', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'region'}, {'feature_label': 'region', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('region')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'crop_type', 'transformation_label': 'String Indexer'}], 'feature': 'crop_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Cotton', 'max': 'Wheat', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'crop_type'}, {'feature_label': 'crop_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('crop_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'irrigation_type', 'transformation_label': 'String Indexer'}], 'feature': 'irrigation_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Drip', 'max': 'Sprinkler', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'irrigation_type'}, {'feature_label': 'irrigation_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('irrigation_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'fertilizer_type', 'transformation_label': 'String Indexer'}], 'feature': 'fertilizer_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Inorganic', 'max': 'Organic', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'fertilizer_type'}, {'feature_label': 'fertilizer_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('fertilizer_type')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'sowing_date', 'transformation_label': 'Extract Date'}], 'feature': 'sowing_date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'sowing_date'}, {'feature_label': 'sowing_date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('sowing_date')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'harvest_date', 'transformation_label': 'Extract Date'}], 'feature': 'harvest_date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'harvest_date'}, {'feature_label': 'harvest_date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('harvest_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'sensor_id', 'transformation_label': 'String Indexer'}], 'feature': 'sensor_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'SENS0001', 'max': 'SENS0500', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sensor_id'}, {'feature_label': 'sensor_id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('sensor_id')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'timestamp', 'transformation_label': 'Extract Date'}], 'feature': 'timestamp', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'timestamp'}, {'feature_label': 'timestamp', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('timestamp')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'crop_disease_status', 'transformation_label': 'String Indexer'}], 'feature': 'crop_disease_status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Mild', 'max': 'Severe', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'crop_disease_status'}, {'feature_label': 'crop_disease_status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('crop_disease_status')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run smartfarmingsensordataforyieldpredictionHooks.ipynb
try:
	#sourcePreExecutionHook()

	smartfarmingcropyield = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Smart_Farming_Crop_Yield_2024.csv', 'filename': 'Smart_Farming_Crop_Yield_2024.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(smartfarmingcropyield)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run smartfarmingsensordataforyieldpredictionHooks.ipynb
try:
	#transformationPreExecutionHook()

	smartfarmingsensordataforyieldpredictionautofe = TransformationMain.run(smartfarmingcropyield,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "farm_id", "transformation_label": "String Indexer"}], "feature": "farm_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "FARM0001", "max": "FARM0500", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "farm_id"}, {"transformationsData": [{"feature_label": "region", "transformation_label": "String Indexer"}], "feature": "region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Central USA", "max": "South USA", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "region"}, {"transformationsData": [{"feature_label": "crop_type", "transformation_label": "String Indexer"}], "feature": "crop_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Cotton", "max": "Wheat", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "crop_type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "soil_moisture_%", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "26.75", "stddev": "10.15", "min": "10.16", "max": "44.98", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "soil_moisture_%"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "soil_pH", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.52", "stddev": "0.59", "min": "5.51", "max": "7.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "soil_pH"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "temperature_C", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "24.68", "stddev": "5.35", "min": "15.0", "max": "34.84", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "temperature_C"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rainfall_mm", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "181.69", "stddev": "72.29", "min": "50.17", "max": "298.96", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "rainfall_mm"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "humidity_%", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "65.19", "stddev": "14.64", "min": "40.23", "max": "90.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "humidity_%"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sunlight_hours", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.03", "stddev": "1.69", "min": "4.01", "max": "10.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "sunlight_hours"}, {"transformationsData": [{"feature_label": "irrigation_type", "transformation_label": "String Indexer"}], "feature": "irrigation_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Drip", "max": "Sprinkler", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "irrigation_type"}, {"transformationsData": [{"feature_label": "fertilizer_type", "transformation_label": "String Indexer"}], "feature": "fertilizer_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Inorganic", "max": "Organic", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "fertilizer_type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pesticide_usage_ml", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "26.59", "stddev": "13.2", "min": "5.05", "max": "49.94", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "pesticide_usage_ml"}, {"transformationsData": [{"feature_label": "sowing_date", "transformation_label": "Extract Date"}], "feature": "sowing_date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "sowing_date"}, {"transformationsData": [{"feature_label": "harvest_date", "transformation_label": "Extract Date"}], "feature": "harvest_date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "harvest_date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_days", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "119.5", "stddev": "16.8", "min": "90", "max": "150", "missing": "0"}, "updatedLabel": "total_days"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "yield_kg_per_hectare", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "4032.93", "stddev": "1174.43", "min": "2023.56", "max": "5998.29", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "yield_kg_per_hectare"}, {"transformationsData": [{"feature_label": "sensor_id", "transformation_label": "String Indexer"}], "feature": "sensor_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "SENS0001", "max": "SENS0500", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sensor_id"}, {"transformationsData": [{"feature_label": "timestamp", "transformation_label": "Extract Date"}], "feature": "timestamp", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "timestamp"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "latitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "22.44", "stddev": "7.28", "min": "10.004243", "max": "34.981531", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "latitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "longitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "80.39", "stddev": "5.91", "min": "70.020021", "max": "89.991901", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "longitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NDVI_index", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.6", "stddev": "0.18", "min": "0.3", "max": "0.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NDVI_index"}, {"transformationsData": [{"feature_label": "crop_disease_status", "transformation_label": "String Indexer"}], "feature": "crop_disease_status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Mild", "max": "Severe", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "crop_disease_status"}]}))

	#transformationPostExecutionHook(smartfarmingsensordataforyieldpredictionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run smartfarmingsensordataforyieldpredictionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(smartfarmingsensordataforyieldpredictionautofe, ["soil_moisture_%", "soil_pH", "temperature_C", "rainfall_mm", "humidity_%", "sunlight_hours", "pesticide_usage_ml", "total_days", "yield_kg_per_hectare", "latitude", "longitude", "NDVI_index", "farm_id_stringindexer", "region_stringindexer", "crop_type_stringindexer", "irrigation_type_stringindexer", "fertilizer_type_stringindexer", "sowing_date_dayofmonth", "sowing_date_month", "sowing_date_year", "harvest_date_dayofmonth", "harvest_date_month", "harvest_date_year", "sensor_id_stringindexer", "timestamp_dayofmonth", "timestamp_month", "timestamp_year"], "crop_disease_status_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

